# L'importance des modèles

## Données

Repartons des données que nous avons mis en forme, et amenons progressivement une analyse plus avancée

In [1]:
import pandas as pd
df = pd.read_excel("../data/data_hal_crest_clean.xlsx")
df.head()

,Unnamed: 0,docid,title,abstract,abstract_eng,publicationDate_tdate,document_type,authors,date,abstract_net,title_net,texte,taille,cont_IA,cont_algo,cont_ml
0,0,3026577,"[""The effect of price magnitude on analysts' f...",['Recent research in finance shows that the ma...,['Recent research in finance shows that the ma...,2021-01-01T00:00:00Z,shs.eco,"['Tristan Roger', 'Wael Bousselmi', 'Patrick R...",2021-01-01,Recent research in finance shows that the magn...,The effect of price magnitude on analysts' for...,The effect of price magnitude on analysts' for...,822,False,False,False
1,1,4358212,['Model-based Clustering with Missing Not At R...,"['Model-based unsupervised learning, as any le...","['Model-based unsupervised learning, as any le...",2024-06-18T00:00:00Z,stat.ml,"['Aude Sportisse', 'Matthieu Marbac', 'Fabien ...",2024-06-18,"Model-based unsupervised learning, as any lear...",Model-based Clustering with Missing Not At Ran...,Model-based Clustering with Missing Not At Ran...,1409,False,True,False
2,2,4996040,['Inferring the parameters of Taylor’s power l...,['Taylor’s power law of fluctuation scaling ha...,['Taylor’s power law of fluctuation scaling ha...,2025-11-26T00:00:00Z,stat,"['Lionel Truquet', 'Joel E. Cohen', 'Paul Douk...",2025-11-26,Taylor’s power law of fluctuation scaling has ...,Inferring the parameters of Taylor’s power law...,Inferring the parameters of Taylor’s power law...,1570,False,False,False
3,3,5357551,['Nonparametric intensity estimation of spatia...,['We propose a random forest estimator for the...,['We propose a random forest estimator for the...,2025-11-10T00:00:00Z,stat.me,"['Christophe Biscio', 'Frédéric Lavancier']",2025-11-10,We propose a random forest estimator for the i...,Nonparametric intensity estimation of spatial ...,Nonparametric intensity estimation of spatial ...,814,False,False,False
4,4,3026615,['Impact of the Brexit vote announcement on lo...,['We examine how the Brexit announcement influ...,['We examine how the Brexit announcement influ...,2020-01-01T00:00:00Z,shs.eco,"['Wael Bousselmi', 'Patrick Sentis', 'Marc Wil...",2020-01-01,We examine how the Brexit announcement influen...,Impact of the Brexit vote announcement on long...,Impact of the Brexit vote announcement on long...,759,False,False,False


## Spacy pour aller vers les modèles

Installer Spacy et un modèle adapté

In [ ]:
#!pip install -U spacy
#!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 792.7 kB/s  0:00:380:00:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [2]:
import spacy
pipeline = spacy.load("en_core_web_md")
doc = pipeline("The language Python rocks, but the day is long.")

In [3]:
type(doc)

spacy.tokens.doc.Doc

Le texte est "structuré" : composé de différents éléments qui n'existaient pas avant

In [4]:
doc[0]

The

POS, lemmatisation, etc.

In [14]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.lemma_, token.is_oov)

The the DET det the False
language language NOUN compound language False
Python Python PROPN compound Python False
rocks rock NOUN ROOT rock False
, , PUNCT punct , False
but but CCONJ cc but False
the the DET det the False
day day NOUN nsubj day False
is be AUX conj be False
long long ADJ acomp long False
. . PUNCT punct . False


Récupérer uniquement les verbes ?

NER

In [15]:
for ent in doc.ents:
    print(ent.text, ent.label_)

the day DATE


In [16]:
result = df["texte"][0:10].apply(pipeline)

In [20]:
for ent in result.iloc[0].ents:
    print(ent.text, ent.label_)

Roger et al. PERSON
2018 DATE
two CARDINAL
one CARDINAL
one CARDINAL


In [5]:
from spacy import displacy
displacy.render(doc, style="ent", 
                jupyter=True)

Plongements

In [11]:
# Compare two documents
doc1 = pipeline(df["texte"][0])
doc2 = pipeline(df["texte"][1000])
print(doc1.similarity(doc2))

0.6429994113259276


## Utiliser des modèles d'HuggingFace : le cas de GliNER

Aller vers les modèles :
- HuggingFace
- Cartes des modèles
- Transformers

Un modèle plus complexe pour l'identification d'entités

Par exemple : https://huggingface.co/urchade/gliner_multi_pii-v1

In [ ]:
#!pip install gliner
from gliner import GLiNER

model = GLiNER.from_pretrained("gliner-community/gliner_large-v2.5")

/opt/anaconda3/envs/pyshs/lib/python3.12/site-packages/huggingface_hub/utils/_validators.py:186: UserWarning: The `resume_download` argument is deprecated and ignored in `snapshot_download`. Downloads always resume whenever possible.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
text = "Il se trouve qu'Émilien aime faire du Python"

labels = ["person"]

entities = model.predict_entities(text, labels)

entities

[{'start': 16,
  'end': 23,
  'text': 'Émilien',
  'label': 'person',
  'score': 0.987253725528717}]

In [27]:
df["texte"][0:10].apply(lambda x:  model.predict_entities(x, ["mathematics"]))

/opt/anaconda3/envs/pyshs/lib/python3.12/site-packages/gliner/data_processing/processor.py:372: UserWarning: Sentence of length 1780 has been truncated to 768
  batch = [self.preprocess_example(b["tokenized_text"], b[key], class_to_ids) for b in batch_list]
/opt/anaconda3/envs/pyshs/lib/python3.12/site-packages/gliner/data_processing/processor.py:372: UserWarning: Sentence of length 954 has been truncated to 768
  batch = [self.preprocess_example(b["tokenized_text"], b[key], class_to_ids) for b in batch_list]


0                                                   []
1    [{'start': 412, 'end': 425, 'text': 'mixture m...
2    [{'start': 28, 'end': 46, 'text': 'Taylor’s po...
3    [{'start': 38, 'end': 61, 'text': 'spatial poi...
4                                                   []
5    [{'start': 202, 'end': 213, 'text': 'source te...
6                                                   []
7                                                   []
8    [{'start': 16, 'end': 36, 'text': 'linear loss...
9    [{'start': 347, 'end': 364, 'text': 'covarianc...
Name: texte, dtype: object

## Application

Faire de l'analyse de sentiments

Une question : **quelles sont les prises de paroles les plus négatives ?**

- Embarras du choix
    - Par ex : [🚀 distilbert-based Multilingual Sentiment Classification Model
](https://huggingface.co/tabularisai/multilingual-sentiment-analysis)
- Comprendre le modèle / ce qu'il fait
- Importance d'évaluer son résultat